
# London Subway Optimization
In this project, we analysed the flow capacity of the London Subway System in many ways. The goal of the project was to propose the most cost-effective improvement to maximise the commuter flow in the network, and was mostly solved using the Gurobi Python library.

For this project we will use data on the London subway. The data is organized as follows:
- file ''connections'' contains data on the direct links between two stations; for instance, we can go from Station 11 to Station 163 (and vice versa) by taking line 1; it will take us 1 minute;
- file ''stations'' contains information on identifiers of each station;
- file ''stations_restricted'' contains a subset of the stations from stations.csv

From this information, we can construct a network as follows:
- the nodes are the stations;
- there is an arc between two nodes if there is a direct link between the two corresponding stations in the file ''connections'' (assume every connection corresponds to two arcs in opposite directions); for instance, there is an arc (11,163), and arc (163,11), but no arc (1,170);
- define capacities as follows:
1.   Generate a random number between 5 and 10 for each line: this is the capacity of the line.
2.   The capacity of an arc between two stations has the same capacity as the sum of the capacities of the lines that link them. For instance, the stations of King's Cross St. Pancras (#145) and Euston Square (#90) are linked by lines 3,6,8. Let's say we randomly sampled the capacity of 6,8,4 for lines 3,6,8. The capacity of the arc (145,90) is 6+8+4=18. Similarly, the capacity of the arc (90,145) is also 6+8+4=18.

The project is composed of 4 questions, all to be solved using Linear Program and to be completed in this order:

1. We want to compute how many passengers can transit on average from the north-east part of the city center, to the south-west. To do so, compute a maximum flow between King's Cross St. Pancras (#145) and Hammersmith (#110) on the instance in the file ''connections'' (ignore the time it takes to go between stations).
2. We want to understand how many people can be evacuated from the city center within 18 minutes. To do so, restrict to the stations in the file ''stations_restricted'' (and the corresponding arcs). Compute the maximum flow over time between King's Cross St. Pancras (#145) and Victoria (#273) for T=18.
3. Consider again the restriction to the stations in the file ''stations_restricted''. For security reasons, in the stations Oxford Circus (#192) and Embankment (#87), at any given time there can be at most 20 passengers in transit. Solve problem 2) above under this additional restriction.
4. **4) Will not be evaluated unless parts 1),2),3) have been correctly solved** Choose an improvement to the line and write a linear program that finds the *best* way to realize your improvement. For instance, you could assume you have a budget of 10 units of additional capacity to be added to the network, and you want to decide where to add them so that the maximum flow in part 1) above increases the most (it is ok to have an improvement that requires an integer solution, but the solution you get is fractional).





#Data Input#

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
%pip install gurobipy
import gurobipy as gp

Upload and inspection of first data set (connections)

In [ ]:
# Set working directory
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Read connections File
df = pd.read_csv('/content/drive/My Drive/Project 1 Optimization/connections.csv')
print(df.head())

# Read stations list file
stations_list = pd.read_csv('/content/drive/My Drive/Project 1 Optimization/stations.csv')
print(stations_list.head())

   link name  station1  station2  line  time  distance(miles)
0        338         1        73    10   2.0         0.612553
1        339         1       234    10   4.0         1.170653
2        340         1       265    10   3.0         1.214326
3         73         2       156     3   2.0         0.379161
4         74         2       263     3   4.0         0.314278
   Station code          name  zone  total_lines
0             1    Acton Town   3.0          2.0
1             2       Aldgate   1.0          2.0
2             3  Aldgate East   1.0          2.0
3             4    All Saints   2.0          1.0
4             5      Alperton   4.0          1.0


Generate random capacities for each line and add them to df

In [ ]:
# Set seed to make it reproducable
np.random.seed(14)

# For each unique line generate a random (integer) capacity from 5 to 10
lines = df['line'].unique()
lines_capacities = pd.DataFrame({'line': lines, 'line_capacity': np.random.randint(5, 11, len(lines))})
print (lines_capacities)

# Add a column to df with the capacity of the line
df = pd.merge(df, lines_capacities, how='left', on='line')

    line  line_capacity
0     10              8
1      3              5
2      8              9
3      4              6
4      6              7
5     13              9
6      9              7
7      1              5
8      7             10
9      2              5
10    12             10
11    11              7
12    14              6
13     5              8


In [ ]:
df.head()

,link name,station1,station2,line,time,distance(miles),line_capacity
0,338,1,73,10,2.0,0.612553,8
1,339,1,234,10,4.0,1.170653,8
2,340,1,265,10,3.0,1.214326,8
3,73,2,156,3,2.0,0.379161,5
4,74,2,263,3,4.0,0.314278,5


Capacities for each arc between two stations

In [ ]:
# We will construct a dictionary where the value of (i, j) will correspond to the total capacity of the arc connecting i and j
# Initialize dictionary
c = {}

# Loop to get the total capacity of an arc connecting 2 stations by adding the capacities of all the lines that connect them
for index, row in df.iterrows():
  s1 = int(row['station1'])
  s2 = int(row['station2'])
  capacity_line = int(row['line_capacity'])
  # Add capcity of new link between s1 and s2 to total capacity of arc s1 -> s2 (and s2 -> s1, since bidirectional network)
  c[(s1, s2)] = c.get((s1, s2), 0) + capacity_line
  c[(s2, s1)] = c.get((s2, s1), 0) + capacity_line

In [ ]:
print(dict(sorted(c.items(), key=lambda item: (item[0][0], item[0][1]))))

{(1, 52): 6, (1, 73): 14, (1, 234): 8, (1, 265): 8, (2, 156): 14, (2, 263): 5, (3, 156): 7, (3, 263): 6, (3, 295): 13, (4, 70): 9, (4, 201): 9, (5, 194): 8, (5, 252): 8, (6, 46): 9, (7, 145): 7, (7, 188): 7, (8, 124): 7, (8, 264): 7, (9, 31): 8, (9, 232): 8, (10, 95): 8, (10, 128): 8, (11, 28): 10, (11, 83): 12, (11, 94): 9, (11, 104): 21, (11, 163): 5, (11, 212): 5, (11, 249): 10, (12, 56): 7, (12, 257): 7, (13, 156): 5, (13, 157): 7, (13, 167): 7, (13, 225): 9, (13, 250): 5, (13, 279): 10, (14, 92): 21, (14, 167): 21, (15, 78): 13, (15, 269): 6, (16, 91): 5, (16, 173): 5, (17, 74): 8, (17, 110): 14, (17, 293): 6, (18, 186): 11, (18, 193): 11, (19, 97): 9, (20, 65): 9, (20, 217): 9, (21, 67): 6, (21, 269): 6, (22, 47): 7, (22, 111): 7, (23, 41): 10, (23, 157): 10, (24, 156): 5, (24, 164): 5, (25, 161): 11, (25, 255): 11, (26, 260): 7, (26, 274): 7, (27, 79): 9, (27, 201): 9, (28, 11): 10, (28, 107): 10, (28, 162): 5, (28, 192): 5, (28, 193): 6, (28, 259): 6, (29, 84): 7, (29, 157): 7,

#Problem 1:#
We want to solve the max flow problem between Hammersmith (#110) and King's Cross St. Pancras (#145). The max flow is the same in both directions as the network is undirected

In [ ]:
# Fixed parameters
stations = set(df['station1']).union(set(df['station2'])) # List all unique stations
source = 110
sink = 145

# Initialize a LP model
m1 = gp.Model()

# Variables: x[i, j] >= 0 representing the flow frow station i to j
# One variable for each arc with positive capacity
x = {}
for (s1, s2), capacity in c.items():
  x[(s1, s2)] = m1.addVar(lb=0, ub=capacity, vtype=gp.GRB.INTEGER) # Note we already included the non-negativity and capacity constraints

# Add conservation of flow constraint
for s in stations:
  if s == source or s == sink: continue
  else:
    # flow entering a node = flow leaving it
    m1.addConstr(sum(x[(n, s)] for n in stations if (n, s) in x) == sum(x[(s, n)] for n in stations if (s, n) in x))

# Set objective function: maximize net flux arriving at Hammersmith
# Note: maximizing net flux leaving King's cross would be equivalent
m1.setObjective(sum(x[(n, sink)] - x[(sink, n)] for n in stations if (n, sink) in x), gp.GRB.MAXIMIZE)

# Optimize the model
m1.optimize()


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 320 rows, 752 columns and 1482 nonzeros
Model fingerprint: 0x8b86761a
Variable types: 0 continuous, 752 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e+00, 2e+01]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 286 rows and 674 columns
Presolve time: 0.00s
Presolved: 34 rows, 78 columns, 146 nonzeros
Variable types: 0 continuous, 78 integer (0 binary)
Found heuristic solution: objective 1.0000000

Root relaxation: objective 3.400000e+01, 51 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

In [ ]:
# Output of the results
print(f"\nMaximum flow from station 145 to 110: {m1.objVal}")
for (st1, st2), x_opt in x.items():
  if x_opt.X > 0:  # We only care about positive flows
    print(f"Flow from station {st1} to station {st2}: {x_opt.X}")


Maximum flow from station 145 to 110: 34.0
Flow from station 1 to station 73: 14.0
Flow from station 73 to station 1: 1.0
Flow from station 265 to station 1: 8.0
Flow from station 2 to station 156: 14.0
Flow from station 156 to station 2: 9.0
Flow from station 263 to station 2: 5.0
Flow from station 3 to station 263: 6.0
Flow from station 295 to station 3: 13.0
Flow from station 3 to station 156: 7.0
Flow from station 4 to station 70: 9.0
Flow from station 70 to station 4: 4.0
Flow from station 201 to station 4: 5.0
Flow from station 194 to station 5: 8.0
Flow from station 5 to station 252: 8.0
Flow from station 7 to station 145: 7.0
Flow from station 188 to station 7: 7.0
Flow from station 95 to station 10: 7.0
Flow from station 10 to station 128: 7.0
Flow from station 212 to station 11: 5.0
Flow from station 83 to station 11: 12.0
Flow from station 11 to station 104: 21.0
Flow from station 11 to station 28: 10.0
Flow from station 28 to station 11: 6.0
Flow from station 11 to station

#Problem 2:#
How many people can be evacuated from #145 to #273 within 18 minutes.

Read *stations_restricted.csv* to find out what stations to consider for this problem (and problem 3)

In [ ]:
# Read stations list file
stations_restricted = pd.read_csv('/content/drive/My Drive/Project 1 Optimization/stations_restricted.csv')

# Create new data frame = subset of df (where stations in stations_restricted)
df_restricted = df[
    df['station1'].isin(stations_restricted['Station code']) &
    df['station2'].isin(stations_restricted['Station code'])
]

In [ ]:
# Create a dictionary that for every connection i to j it returns the duration of the ride
duration = {}
for index, row in df_restricted.iterrows():
  s1 = int(row['station1'])
  s2 = int(row['station2'])
  # Check for NA in time
  if pd.isna(row['time']):
    print(f'Connection between {s1} and {s2} has an undefined duration! We assign a time =  1 min')
    duration_line = 1 # if NA we will (arbitrarily) assign it a value of 1
  else: duration_line = int(row['time'])

  if duration.get((s1, s2)) or duration.get((s2, s1)):
    if duration[(s1, s2)] != duration_line or duration[(s1, s2)] != duration_line:
      print(f'There are connections between {s1} and {s2} with different duration times')
  else:
    duration[(s1, s2)] = duration_line
    duration[(s2, s1)] = duration_line

Connection between 259 and 28 has an undefined duration! We assign a time =  1 min


In [ ]:
print(duration)
print(max(duration.values()), min(duration.values()))

{(11, 212): 2, (212, 11): 2, (11, 104): 3, (104, 11): 3, (11, 28): 2, (28, 11): 2, (28, 192): 1, (192, 28): 1, (28, 107): 2, (107, 28): 2, (49, 87): 1, (87, 49): 1, (49, 197): 2, (197, 49): 2, (49, 151): 2, (151, 49): 2, (60, 126): 1, (126, 60): 1, (60, 151): 1, (151, 60): 1, (87, 285): 2, (285, 87): 2, (89, 145): 2, (145, 89): 2, (89, 277): 1, (277, 89): 1, (90, 104): 2, (104, 90): 2, (90, 145): 2, (145, 90): 2, (102, 259): 1, (259, 102): 1, (102, 277): 2, (277, 102): 2, (107, 285): 3, (285, 107): 3, (107, 197): 1, (197, 107): 1, (107, 192): 2, (192, 107): 2, (107, 273): 2, (273, 107): 2, (126, 259): 2, (259, 126): 2, (126, 223): 2, (223, 126): 2, (145, 223): 2, (223, 145): 2, (151, 259): 1, (259, 151): 1, (151, 197): 2, (197, 151): 2, (192, 197): 2, (197, 192): 2, (192, 212): 2, (212, 192): 2, (192, 259): 2, (259, 192): 2, (192, 277): 2, (277, 192): 2, (248, 273): 2, (273, 248): 2, (248, 285): 2, (285, 248): 2, (259, 28): 1, (28, 259): 1}
3 1


In [ ]:
# Fixed parameters
stations_rest = stations_restricted['Station code'].unique() # list of all unique restricted stations
origin = 145
destination = 273
T = 18 # time for evacuation

# Initialize a LP model
m2 = gp.Model()

# Variables: y(s1, s2, t) represents the flow leaving from s1 towards s2 at instant t
y = {}
for (s1, s2), d in duration.items():
  for t in range(T+1):
    if t + d <= T:
      y[(s1, s2, t)] = m2.addVar(lb=0, ub=c[((s1, s2))], vtype=gp.GRB.INTEGER) # Note we already included the non-negativity and capacity constraints

# Add constraints
# Conservation of flow constraint subject to time-constraint
for s in stations_rest:
  if s == origin or s == destination: continue
  else:
    # flow entering a node = flow leaving it at any given time
    for t in range(T+1):
      m2.addConstr(sum(y.get((s, n, t), 0) for n in stations_rest) == \
                   sum(y.get((n, s, t - duration.get((n, s), 0)), 0) for n in stations_rest))

# Set objective function: maximize net flux arriving at Victoria during the 18 minutes
obj_fun_2 = sum(y.get((n, destination, t), 0) - y.get((destination, n, t), 0)
                for n in stations_rest for t in range(T+1))

m2.setObjective(obj_fun_2, gp.GRB.MAXIMIZE)

# Optimize the model
m2.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 380 rows, 1136 columns and 2102 nonzeros
Model fingerprint: 0x98bec04a
Variable types: 0 continuous, 1136 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e+00, 2e+01]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 273 rows and 790 columns
Presolve time: 0.01s
Presolved: 107 rows, 346 columns, 650 nonzeros
Found heuristic solution: objective 14.0000000
Variable types: 0 continuous, 346 integer (0 binary)
Found heuristic solution: objective 21.0000000

Root relaxation: objective 1.100000e+02, 123 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      | 

In [ ]:
# Output of the results
print(f"\nMaximum flow for evacuation under 18 min from station 145 to 273: {m2.objVal}")

for (st1, st2, t), y_opt in sorted(y.items(), key=lambda item: item[0][2]):  # Sort by t
    if y_opt.X > 0:  # Only care about positive flows
        print(f"Flow from station {st1} to station {st2} @ instant {t}: {y_opt.X}")


Maximum flow for evacuation under 18 min from station 145 to 273: 110.0
Flow from station 145 to station 89 @ instant 0: 14.0
Flow from station 145 to station 90 @ instant 0: 10.0
Flow from station 145 to station 223 @ instant 0: 8.0
Flow from station 145 to station 89 @ instant 1: 14.0
Flow from station 145 to station 90 @ instant 1: 10.0
Flow from station 145 to station 223 @ instant 1: 8.0
Flow from station 145 to station 89 @ instant 2: 14.0
Flow from station 89 to station 277 @ instant 2: 14.0
Flow from station 90 to station 104 @ instant 2: 10.0
Flow from station 145 to station 90 @ instant 2: 2.0
Flow from station 223 to station 126 @ instant 2: 8.0
Flow from station 145 to station 223 @ instant 2: 1.0
Flow from station 145 to station 89 @ instant 3: 7.0
Flow from station 89 to station 277 @ instant 3: 14.0
Flow from station 90 to station 104 @ instant 3: 10.0
Flow from station 145 to station 90 @ instant 3: 2.0
Flow from station 277 to station 102 @ instant 3: 7.0
Flow from st

#Problem 3:#
Same evacuation problem as P2, but additionally: for security reasons, in the stations Oxford Circus (#192) and Embankment (#87), at any given time there can be at most 20 passengers in transit.

In [ ]:
# Parameters
# Note: same fixed parameters as before +
oxford = 192
embankment = 87
max_people_at_station = 20

# Initialize a LP model
m3 = gp.Model()

# Variables: z(s1, s2, t) represents the flow leaving from s1 towards s2 at instant t
z = {}
for (s1, s2), d in duration.items():
  for t in range(T+1):
    if t + d <= T:
      z[(s1, s2, t)] = m3.addVar(lb=0, ub=c[((s1, s2))], vtype=gp.GRB.INTEGER) # Note we already included the non-negativity and capacity constraints

# Add constraints
# 1) Security constraint for #192 to #87
# 2) Conservation of flow constraint subject to time-constraint
for s in stations_rest:
  if s == oxford or s == embankment:
     for t in range(T+1):
      m3.addConstr(sum(z.get((s, n, t), 0) for n in stations_rest) <= max_people_at_station)
  if s == origin or s == destination: continue
  else:
    # flow entering a node = flow leaving it at any given time
    for t in range(T+1):
      m3.addConstr(sum(z.get((s, n, t), 0) for n in stations_rest) == \
                   sum(z.get((n, s, t - duration.get((n, s), 0)), 0) for n in stations_rest))

# Set objective function: maximize net flux arriving at Victoria during the 18 minutes
obj_fun_3 = sum(z.get((n, destination, t), 0) - z.get((destination, n, t), 0)
                for n in stations_rest for t in range(T+1))

m3.setObjective(obj_fun_3, gp.GRB.MAXIMIZE)

# Optimize the model
m3.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 418 rows, 1136 columns and 2240 nonzeros
Model fingerprint: 0x5b6cd8d3
Variable types: 0 continuous, 1136 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e+00, 2e+01]
  RHS range        [2e+01, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 299 rows and 781 columns
Presolve time: 0.01s
Presolved: 119 rows, 355 columns, 712 nonzeros
Found heuristic solution: objective 14.0000000
Variable types: 0 continuous, 355 integer (0 binary)
Found heuristic solution: objective 21.0000000

Root relaxation: objective 1.100000e+02, 111 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      | 

In [ ]:
# Output of the results
print(f"\nMaximum flow for evacuation under 18 min from station 145 to 273 (under addtional constraints): {m3.objVal}")

for (st1, st2, t), z_opt in sorted(z.items(), key=lambda item: item[0][2]):  # Sort by t
    if z_opt.X > 0:  # Only care about positive flows
        print(f"Flow from station {st1} to station {st2} @ instant {t}: {z_opt.X}")


Maximum flow for evacuation under 18 min from station 145 to 273 (under addtional constraints): 110.0
Flow from station 145 to station 89 @ instant 0: 14.0
Flow from station 145 to station 90 @ instant 0: 21.0
Flow from station 273 to station 107 @ instant 0: 7.0
Flow from station 145 to station 223 @ instant 0: 8.0
Flow from station 273 to station 248 @ instant 0: 11.0
Flow from station 145 to station 89 @ instant 1: 14.0
Flow from station 145 to station 90 @ instant 1: 21.0
Flow from station 273 to station 107 @ instant 1: 7.0
Flow from station 145 to station 223 @ instant 1: 8.0
Flow from station 273 to station 248 @ instant 1: 11.0
Flow from station 145 to station 89 @ instant 2: 14.0
Flow from station 89 to station 277 @ instant 2: 14.0
Flow from station 90 to station 104 @ instant 2: 21.0
Flow from station 145 to station 90 @ instant 2: 21.0
Flow from station 107 to station 285 @ instant 2: 7.0
Flow from station 223 to station 126 @ instant 2: 8.0
Flow from station 248 to statio

#Problem 4:#

**Improvement Idea 1:** capacity improvement for 3 connections.

In [ ]:
# P4: +2 Capacity improvements on 3 connections.

# Suppose we have the budget to improve 3 of the connections i, j in problem 1 by 2 units.
# We have to choose which lines to improve based on which is going to most increment the overall flow from source to sink.

# Fixed parameters
stations = set(df['station1']).union(set(df['station2'])) # List all unique stations
source = 110
sink = 145

# Initialize a LP model
m4_cap = gp.Model()

# Variables: x[i, j] >= 0 representing the flow from station i to j
# One variable for each arc with positive capacity
# One binary variable for each arc to account for the improvement.

x = {}
imp = {}

for (s1, s2), capacity in c.items():
  x[(s1, s2)] = m4_cap.addVar(lb=0, vtype=gp.GRB.INTEGER) # Note we already included the non-negativity constraints.
  imp[(s1, s2)] = m4_cap.addVar(vtype=gp.GRB.BINARY)

# New Capacity Constraints
for (s1, s2), capacity in c.items():
  m4_cap.addConstr(x[(s1, s2)] <= capacity + 2 * imp[(s1, s2)]) #Add 2 capacity units if choose to improve

#improve at most 3 connections
m4_cap.addConstr(sum(imp[(n1, n2)] for (n1, n2), capacity in c.items()) <= 3)

# Add conservation of flow constraint
for s in stations:
  if s == source or s == sink: continue
  else:
    # flow entering a node = flow leaving it
    m4_cap.addConstr(sum(x[(n, s)] for n in stations if (n, s) in x) == sum(x[(s, n)] for n in stations if (s, n) in x))

# Set objective function: maximize net flux arriving at Hammersmith
# Note: maximizing net flux leaving King's cross would be equivalent
m4_cap.setObjective(sum(x[(n, sink)] - x[(sink, n)] for n in stations if (n, sink) in x), gp.GRB.MAXIMIZE)

# Optimize the model
m4_cap.optimize()


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 1073 rows, 1504 columns and 3738 nonzeros
Model fingerprint: 0x3a2ced71
Variable types: 0 continuous, 1504 integer (752 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 662 rows and 944 columns
Presolve time: 0.02s
Presolved: 411 rows, 560 columns, 1466 nonzeros
Variable types: 0 continuous, 560 integer (364 binary)

Root relaxation: objective 3.628571e+01, 238 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

 

In [ ]:
# Output of Results
print (f"\nMaximum flow from station 145 to 110 with improvements:  {m4_cap.objVal}")
print ("\nImproved Connections:")
for (n1, n2), var in imp.items():
  if var.X == 1:
    print(f"Connection from station {n1} to station {n2}")


Maximum flow from station 145 to 110 with improvements:  36.0

Improved Connections:
Connection from station 110 to station 17
Connection from station 17 to station 74


 **Improvement Idea 2:** choose 1 line to improve by 2 capacity units. (This could equal an improvement of the trains for a certain line)

In [ ]:
#P4: +2 capacity on Line improvement (new trains for line X)

c = {}

#Store all the capacities of the links and their respective lines.

for index, row in df.iterrows():

  s1 = int(row['station1'])
  s2 = int(row['station2'])
  line_id = row['line']
  capacity_line = int(row['line_capacity'])

  if (s1, s2) not in c:
        c[(s1, s2)] = {}
        c[(s2, s1)] = {}

  # This time, we store the information more granularly by saving each link, and not adding up every capacity yet.
  c[(s1, s2)][line_id] = capacity_line
  c[(s2, s1)][line_id] = capacity_line

stations = set(df['station1']).union(set(df['station2'])) # List all unique stations
source = 110
sink = 145

# Initialize a LP model
m4_lin = gp.Model()

# Variables: x[i, j] >= 0 representing the flow frow station i to j

# One variable for each arc with positive capacity
x = {}
for (s1, s2), capacity in c.items():
  x[(s1, s2)] = m4_lin.addVar(lb=0, vtype=gp.GRB.INTEGER) # Note we already included the non-negativity and capacity constraints

# Variables for which line to improve
y = {}
for line in lines:
    y[line] = m4_lin.addVar(vtype=gp.GRB.BINARY)

# Capacity constraints keeping in mind the line improvement
for (s1, s2), capacities in c.items():
    m4_lin.addConstr(
        x[(s1, s2)] <= sum(cap + (2 * y[line]) for line, cap in capacities.items())
    )

#Improve at most 1 line
m4_lin.addConstr(sum(y[line] for line in lines) <=1)

# Add conservation of flow constraint
for s in stations:
  if s == source or s == sink: continue
  else:
    # flow entering a node = flow leaving it
    m4_lin.addConstr(sum(x[(n, s)] for n in stations if (n, s) in x) == sum(x[(s, n)] for n in stations if (s, n) in x))

# Set objective function: maximize net flux arriving at Hammersmith
# Note: maximizing net flux leaving King's cross would be equivalent
m4_lin.setObjective(sum(x[(n, sink)] - x[(sink, n)] for n in stations if (n, sink) in x), gp.GRB.MAXIMIZE)

# Optimize the model
m4_lin.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 1073 rows, 766 columns and 3114 nonzeros
Model fingerprint: 0x804b5a1f
Variable types: 0 continuous, 766 integer (14 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 832 rows and 556 columns
Presolve time: 0.01s
Presolved: 241 rows, 210 columns, 828 nonzeros
Variable types: 0 continuous, 210 integer (14 binary)

Root relaxation: objective 3.700000e+01, 119 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0

In [ ]:
#Output of Results
print("\nThe new max flow from Station 143 to Station 110 is: ", m4_lin.ObjVal)

for line in lines:
   if y[line].x > 0.5: print(f"\nNew trains for line {line}")


The new max flow from Station 143 to Station 110 is:  37.0

New trains for line 10
